<a href="https://colab.research.google.com/github/pattycorporativo/CursoPO/blob/main/Curso_PO_AVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pesquisa Operacional

##Instalando o módulo python do gurobi

In [3]:
pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 69.6 MB/s eta 0:00:00


## Executando um exemplo

In [3]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# This example formulates and solves the following simple MIP model:
#  maximize
#        x +   y + 2 z
#  subject to
#        x + 2 y + 3 z <= 4
#        x +   y       >= 1
#        x, y, z binary

import gurobipy as gp
from gurobipy import GRB

try:
    # Create a new model
    m = gp.Model("mip1")

    # Create variables
    x = m.addVar(vtype=GRB.BINARY, name="x")
    y = m.addVar(vtype=GRB.BINARY, name="y")
    z = m.addVar(vtype=GRB.BINARY, name="z")

    # Set objective
    m.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, "c0")

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, "c1")

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print(f"{v.VarName} {v.X:g}")

    print(f"Obj: {m.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x98886187
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00

# Exercício Aula 1

## a) Propor um modelo matemático para definir os locais de instalação de postos de corpo de bombeiros.


In [9]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# Exemplo da aula para localização de bases


import gurobipy as gp
import numpy as np
from gurobipy import GRB

try:
    # Create a new model
    model = gp.Model("mip1")

    #[0,   0,   0,   0,   0,   0,   0,   0,   0,   0],

    A = [
   # 1    2    3     4   5    6    7    8    9    10
    [0,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [1,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [1,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [0,   1,   0,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   0,   0,   0,   0,   0,   1,   0,   0],
    [0,   0,   0,   1,   0,   1,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   0,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   1,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   0,   1,   0,   0,   0],
    [0,   0,   0,   0,   0,   0,   0,   1,   1,   0],
    [0,   0,   0,   0,   0,   1,   0,   0,   1,   0],
    [0,   0,   0,   0,   1,   1,   0,   0,   0,   0],
    [0,   0,   0,   0,   1,   0,   1,   0,   0,   1],
    [0,   0,   0,   0,   0,   0,   0,   1,   1,   0],
    [0,   0,   0,   0,   0,   0,   0,   0,   1,   1],
    [0,   0,   0,   0,   0,   0,   0,   0,   0,   1],
    ]

    populacao= np.array([5200, 4400, 7100, 9000, 6100, 5700, 10000,
                         12200, 7600, 20300, 30400, 30900,
                         12000, 9300, 15500, 25600, 11000, 5300, 7900, 9900])

    custo = np.array([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])

    n = len(custo)
    m = len(populacao)

#  minimizar
#        sum c[j] * x[j]
#  subject to
#        sum A[i,j] * x[j] >= 1,  for i = 0, ..., m-1
#               x[j] = {0,1}


    # Create a new model
    model = gp.Model("aula1")

    # Create variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    y = model.addVars(n, vtype=GRB.BINARY, name="x")

    # Set objective
    #model.setObjective(gp.quicksum(custo[j] * x[j] for j in range(n)), GRB.MINIMIZE)
    model.setObjective(gp.quicksum(x[j] for j in range(n)), GRB.MINIMIZE)

    # Add Constraints
    for i in range(m):
        model.addConstr(gp.quicksum(A[i][j] * x[j] for j in range(n)) >= 1)

    # Optimize model
    model.optimize()

    for j in range(n):
       # print(f"{x[j].VarName} = {x[j].X:g}")
       print(f"x[{j+1}] = {x[j].X:g}")

    print(f"Obj: {model.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 20 rows, 20 columns and 37 nonzeros
Model fingerprint: 0x8f584f96
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 20 rows and 20 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.000000000000e+00, best bound 6.000000000000e+00, gap 0.0000%
x[1] = 0
x[2] = 1
x[3] = 1
x[4] = 0
x[5] = 1
x[6] =

### Modelagem q não deu certo

In [2]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# Exemplo da aula para localização de bases


import gurobipy as gp
import numpy as np
from gurobipy import GRB

try:
    # Create a new model
    model = gp.Model("mip1")

    A = [
   # 1    2    3     4   5    6    7    8    9    10
    [0,   1,   1,   0,   0,   0,   0,   0,   0,   0],
    [1,   1,   0,   0,   0,   1,   1,   0,   0,   0],
    [0,   0,   1,   1,   1,   0,   0,   1,   0,   0],
    [1,   1,   1,   0,   0,   0,   0,   1,   0,   1],
    [1,   0,   0,   0,   1,   1,   0,   0,   1,   1]
    ]


    populacao= np.array([5200, 4400, 7100, 9000, 6100, 5700, 10000,
                         12200, 7600, 20300])

    custo = np.array([100, 200, 300, 400, 500])

    n = len(custo)
    m = len(populacao)
    regioes = range(len(populacao))
    instalacoes = range(len(custo))

#  minimizar
#        sum c[j] * y[j][i]
#  subject to
#        sum A[i,j] * y[j][i] = 1,  for i = 0, ..., m-1
#             y[i,j] <= x[j],  for i = 0, ..., m-1
#               x[j] = {0,1}


    # Create a new model
    model = gp.Model("aula1")

    # Create variables
    x = model.addVars(n, vtype=GRB.BINARY, obj=custo, name="x")
    y = model.addMVar((m, n), vtype=GRB.BINARY, name="y")

    # Set objective
    for j in regioes:
        model.setObjective(gp.quicksum(custo[i] * y[j][i] for i in range(n)),
                           GRB.MINIMIZE)

    # Add Constraints - cobertura - apenas uma instalação para cada regiao
    for j in regioes:
        model.addConstr(gp.quicksum(A[j][i] * y[i][j] for j in instalacoes) >= 1)
        #model.addConstr(gp.quicksum(A[i][j] * y[j][i] for j in range(n)) == 1)

    # Add Constraints - cobertura
    #for i in range(n):
    #    model.addConstr( gp.quicksum(y[j][i] for j in regioes) <= x[i])


    # Add Constraints - cobertura
    #model.addConstr(gp.quicksum(x[i] for i in range(n)) ==  2 )


    # Optimize model
    model.optimize()

    for i in range(n):
        print(f"{x[i].VarName} = {x[i].X:g}")

    print(f"Obj: {model.ObjVal:g}")

    print("Regiões atendidas por cada instalação")

    for j in regioes:
      facilidades=f"Região {j} atendida pelas facilidades:["
      for i in range(n):
         #print(f"y[{j}][{i}] = {y[j][i].X:g} {int(y[j][i].X)==1}")
         if (y[j][i].X==1):
            facilidades = facilidades + f"{i} "
      facilidades = facilidades +"]"
      print( facilidades )
except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

ModuleNotFoundError: No module named 'gurobipy'

## b) No caso de haver restrição orçamentária que não permita garantir o atendimento de todas as regiões e, conhecendo a população 𝑝𝑝𝑗𝑗 de cada região 𝑗𝑗, proponha o modelo para instalação de q postos do corpo de bombeiros.

In [29]:
#!/usr/bin/env python3.11

# Copyright 2025, Gurobi Optimization, LLC

# Exemplo da aula para localização de bases


import gurobipy as gp
import numpy as np
from gurobipy import GRB

try:
    # Create a new model
    model = gp.Model("mip1")

    #[0,   0,   0,   0,   0,   0,   0,   0,   0,   0],

    A = [
   # 1    2    3     4   5    6    7    8    9    10
    [0,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [1,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [1,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   0,   0,   0,   0,   0,   0,   0],
    [0,   1,   0,   0,   0,   0,   0,   0,   0,   0],
    [0,   1,   0,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   1,   1,   0,   0,   0,   0,   0,   0],
    [0,   0,   0,   0,   0,   0,   0,   1,   0,   0],
    [0,   0,   0,   1,   0,   1,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   0,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   1,   0,   0,   0,   0],
    [0,   0,   0,   1,   1,   0,   1,   0,   0,   0],
    [0,   0,   0,   0,   0,   0,   0,   1,   1,   0],
    [0,   0,   0,   0,   0,   1,   0,   0,   1,   0],
    [0,   0,   0,   0,   1,   1,   0,   0,   0,   0],
    [0,   0,   0,   0,   1,   0,   1,   0,   0,   1],
    [0,   0,   0,   0,   0,   0,   0,   1,   1,   0],
    [0,   0,   0,   0,   0,   0,   0,   0,   1,   1],
    [0,   0,   0,   0,   0,   0,   0,   0,   0,   1],
    ]

    populacao= np.array([5200, 4400, 7100, 9000, 6100, 5700, 10000,
                         12200, 7600, 20300, 30400, 30900,
                         12000, 9300, 15500, 25600, 11000, 5300, 7900, 9900])

    custo = np.array([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])

    n = len(custo)
    m = len(populacao)
    regioes = range(len(populacao))
    instalacoes = range(len(custo))

#  maximizar
#        sum P[j] * x[j]
#  subject to
#        sum A[i,j] * x[j] + y[j] >= 1,  for i = 0, ..., m-1
#            sum x[i] < 6 , for i=0,..,n
#               x[j] = {0,1}
#               y[j] = {0,1}


    # Create a new model
    model = gp.Model("aula1")

    # Create variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    y = model.addVars(m, vtype=GRB.BINARY, name="y")

    # Set objective
    #model.setObjective(gp.quicksum(x[j] for j in instalacoes), GRB.MINIMIZE)
    model.setObjective(gp.quicksum(populacao[j] * y[j] for j in regioes), GRB.MINIMIZE)


    # Add Constraints
    for j in regioes:
        model.addConstr(gp.quicksum(A[j][i] * x[i] for i in instalacoes)+y[j] >= 1)

    # Add Constraints
    #for j in instalacoes:
    model.addConstr(gp.quicksum( x[i] for i in instalacoes) < 6 )

    # Optimize model
    model.optimize()

    for j in range(n):
       # print(f"{x[j].VarName} = {x[j].X:g}")
       print(f"x[{j+1}] = {x[j].X:g}")

    print("----------------------------------------------")
    for j in regioes:
       # print(f"{x[j].VarName} = {x[j].X:g}")
       print(f"y[{j+1}] = {y[j].X:g}")

    print(f"Obj: {model.ObjVal:g}")

except gp.GurobiError as e:
    print(f"Error code {e.errno}: {e}")

except AttributeError:
    print("Encountered an attribute error")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 21 rows, 30 columns and 67 nonzeros
Model fingerprint: 0x98086229
Variable types: 0 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+03, 3e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 245400.00000
Presolve removed 8 rows and 9 columns
Presolve time: 0.00s
Presolved: 13 rows, 21 columns, 46 nonzeros
Variable types: 0 continuous, 21 integer (21 binary)
Found heuristic solution: objective 134700.00000

Root relaxation: objective 0.000000e+00, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    